---
title: Dashboard Example
subtitle: Interactive Quarto Dashboards with WebAssembly computation
format: live-dashboard
pyodide:
  packages:
    - numpy
    - scipy
    - matplotlib
    - plotly
    - nbformat
    - ipywidgets
---

## Row

### {.sidebar}

```{ojs}
viewof sRange = Inputs.range([0, 50], {value: 10, step: 0.5, label: "sigma"});
viewof bRange = Inputs.range([0, 20], {value: 2.5, step: 0.2, label: "beta"});
viewof rRange = Inputs.range([0, 50], {value: 28, step: 0.5, label: "rho"});
sigma = debounce(viewof sRange);
beta = debounce(viewof bRange);
rho = debounce(viewof rRange);

// From @mbostock/debouncing-input
function debounce(input, delay = 1000) {
  return Generators.observe(notify => {
    let timer = null;
    let value;
    function inputted() {
      if (timer !== null) return;
      notify(value = input.value);
      timer = setTimeout(delayed, delay);
    }
    function delayed() {
      timer = null;
      if (value === input.value) return;
      notify(value = input.value);
    }
    input.addEventListener("input", inputted), inputted();
    return () => input.removeEventListener("input", inputted);
  });
}
```

### Column

::: {.card height="100%" title="The Lorenz System"}

```{pyodide}
#| edit: false
#| echo: false
#| input:
#|   - sigma
#|   - beta
#|   - rho

import plotly.graph_objects as go
import numpy as np
from scipy import integrate

N = 10
def solve_lorenz(N=10, max_time=4.0, sigma=10.0, beta=8./3, rho=28.0):
  def lorenz_deriv(x_y_z, t0, sigma=sigma, beta=beta, rho=rho):
    x, y, z = x_y_z
    return [sigma * (y - x), x * (rho - z) - y, x * y - beta * z]
  np.random.seed(1)
  x0 = -15 + 30 * np.random.random((N, 3))
  t = np.linspace(0, max_time, int(250*max_time))
  x_t = np.asarray([integrate.odeint(lorenz_deriv, x0i, t) for x0i in x0])
  return x_t

x_t = solve_lorenz(N, sigma = sigma, beta = beta, rho = rho)
fig = go.Figure()
for i in range(N):
  fig.add_trace(
      go.Scatter3d(
          x=x_t[i, :, 0],
          y=x_t[i, :, 1],
          z=x_t[i, :, 2],
          showlegend=False,
          mode='lines',
          line=dict(width=4)
      )
  )
fig.update_layout(
    scene_camera_eye=dict(x=1.5, y=-1.5, z=1.5),
    width=800,
    margin=dict(l=0, r=0, b=0, t=0)
)
fig
```
:::